In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

input\tabular-playground-series-aug-2022\sample_submission.csv
input\tabular-playground-series-aug-2022\test.csv
input\tabular-playground-series-aug-2022\train.csv


In [2]:
train = pd.read_csv('input/tabular-playground-series-aug-2022/train.csv',
                    index_col='id')

display(train)


,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,E,158.95,material_7,material_6,6,9,6,16,4,16.301,...,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131,0
26566,E,146.02,material_7,material_6,6,9,10,12,8,17.543,...,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924,0
26567,E,115.62,material_7,material_6,6,9,1,10,1,15.670,...,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364,0


In [3]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold, train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import warnings
import csv
import joblib

In [4]:

#seperate train data into x and y, preserve useful features
x = train.loc[:, ['loading', 'measurement_17', 'measurement_2', 'attribute_3','attribute_2','measurement_5','measurement_4']] 
y = train['failure']

#add features: if measurement_3 or measurement_5 is missing
x['m3_missing'] = train.measurement_3.isna()
x['m5_missing'] = train.measurement_5.isna()

#add feature: use one-hot encoder to process attribute_0
onehot = OneHotEncoder(categories=[['material_5', 'material_7']],
                    drop='first', sparse=False, handle_unknown='ignore')
onehot.fit(train[['attribute_0']])
x['onehot_att0'] = onehot.transform(train[['attribute_0']])

#fill the missing values in the train data using KNNImputer
imputer = KNNImputer(n_neighbors=10)
imputer.fit(x)
x = imputer.transform(x)

x[:, 0] = np.log1p(x[:, 0])
x = np.column_stack((x, x[:, 3] * x[:, 4]))
x = np.delete(x, [3, 4], axis=1)

#use LogisticRegression from sklearn as model
model = make_pipeline(StandardScaler(), LogisticRegression( C=0.08,solver='liblinear', random_state=1, penalty="l1"))
model.fit(x, y)
joblib.dump(model, 'trained_model.joblib')


['trained_model.joblib']